In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cts_recommender import logging_setup
logging_setup.setup_logging('INFO')

In [ ]:
from cts_recommender.preprocessing import ML_feature_processing
from cts_recommender.settings import get_settings

cfg = get_settings()

processed_df, enriched_df = ML_feature_processing.load_processed_and_enriched_programming(
    data_path_processed=cfg.processed_dir / "programming.parquet", 
    data_path_enriched=cfg.processed_dir / "programming_enriched.parquet"
    )

ML_df = ML_feature_processing.build_X_features(processed_df, enriched_df)


In [14]:
import pandas as pd
from cts_recommender.features.schemas import TARGET_FEATURE
from sklearn.model_selection import train_test_split

X = pd.get_dummies(ML_df.drop(columns=[TARGET_FEATURE]))
y = ML_df[TARGET_FEATURE]

# Random split of data movies and non-movies together
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [16]:
len(X_train.columns)

67